In [2]:
# Config
import os
import json
import geojson
import leafmap.leafmap as leafmap
import requests
from PIL import Image

# Basic
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Monitoring
from tqdm.notebook import tqdm

# IO
from os.path import join, exists, basename, dirname, splitext, expanduser
from glob import glob
from shapely.geometry import Point
from shapely import vectorized

import geopandas as gpd

# Parallel processing
from joblib import Parallel, delayed
import concurrent.futures as cf

import rioxarray as rxr

# Load environment variables
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
region = "region_name" # Put roi (ie"uttar_pradesh", "delhi_airshed","lucknow_airshed","bihar")
download_dir = ""
quarter = "" # Put time of interest ex: 2024q1
PLANET_API_KEY= "" ## Put your planet api key 
print(PLANET_API_KEY)
assert PLANET_API_KEY is not None
shape_path = f"{region}.geojson" ## Shape file path
shape_gdf = gpd.read_file(shape_path)
shape_gdf = shape_gdf.to_crs("EPSG:4326")
print(shape_gdf.crs)

In [ ]:


m = leafmap.Map()
m.add_basemap("HYBRID")
m.add_gdf(shape_gdf, zoom_to_layer=True)
m

In [ ]:
headers = {"Authorization": f"api-key {PLANET_API_KEY}"}
params = {
    "name__contains": quarter,
}

response = requests.get(f"https://api.planet.com/basemaps/v1/mosaics", headers=headers, params=params)
response.json()

In [ ]:
assert len(response.json()["mosaics"]) == 1
mosaic = response.json()["mosaics"][0]
print(mosaic['name'])
print(mosaic['id'])
metadata_save_dir = join(download_dir, "metadata", region, mosaic['name'])
min_lon, min_lat, max_lon, max_lat = shape_gdf.bounds.values[0]
print(min_lon, min_lat, max_lon, max_lat)

In [ ]:
params = {
    "bbox": f"{min_lon},{min_lat},{max_lon},{max_lat}",
    "_page_size": 10000,
}

response = requests.get(f"https://api.planet.com/basemaps/v1/mosaics/{mosaic['id']}/quads", headers=headers, params=params)
quads = response.json()['items']
print("Number of quads:", len(quads))

In [ ]:
shape_gdf.plot(color="none", edgecolor="black")
for quad in tqdm(quads):
    bounds = quad['bbox']
    plt.plot([bounds[0], bounds[2], bounds[2], bounds[0], bounds[0]], [bounds[1], bounds[1], bounds[3], bounds[3], bounds[1]], color="red")

In [ ]:
bboxes = np.array([quad['bbox'] for quad in quads])
print(f"{bboxes.shape=}")


print(bboxes)
top_left_inside = vectorized.contains(shape_gdf.geometry.item(), bboxes[:, 0], bboxes[:, 3])
top_right_inside = vectorized.contains(shape_gdf.geometry.item(), bboxes[:, 2], bboxes[:, 3])
bottom_left_inside = vectorized.contains(shape_gdf.geometry.item(), bboxes[:, 0], bboxes[:, 1])
bottom_right_inside = vectorized.contains(shape_gdf.geometry.item(), bboxes[:, 2], bboxes[:, 1])
one_of_the_corners_inside = np.logical_or(np.logical_or(top_left_inside, top_right_inside), np.logical_or(bottom_left_inside, bottom_right_inside))
print(f"{one_of_the_corners_inside.sum()=}")
quads_within_roi = np.array(quads)[one_of_the_corners_inside]
quads_within_roi.shape

In [ ]:
shape_gdf.plot(color="none", edgecolor="black")
for quad in quads_within_roi:
    bounds = quad['bbox']
    plt.plot([bounds[0], bounds[2], bounds[2], bounds[0], bounds[0]], [bounds[1], bounds[1], bounds[3], bounds[3], bounds[1]], color="red")

In [ ]:
# each item in quads_within_roi is a dict. Combine into a geojson feature collection
features = []
for quad in tqdm(quads_within_roi):
    feature = {
        "type": "Feature",
        "geometry": {
            "type": "Polygon",
            "coordinates": [[
                [quad['bbox'][0], quad['bbox'][1]],
                [quad['bbox'][2], quad['bbox'][1]],
                [quad['bbox'][2], quad['bbox'][3]],
                [quad['bbox'][0], quad['bbox'][3]],
                [quad['bbox'][0], quad['bbox'][1]],
            ]],
        },
        "properties": {
            "_self": quad['_links']['_self'],
            "download": quad['_links']['download'],
            "items": quad['_links']['items'],
            "thumbnail": quad['_links']['thumbnail'],
            "id": quad['id'],
            "percent_covered": quad['percent_covered'],
        }
    }
    features.append(feature)
    
collection = geojson.FeatureCollection(features)

In [ ]:
os.makedirs(metadata_save_dir, exist_ok=True)
with open(join(metadata_save_dir, "metadata.geojson"), "w") as f:
    geojson.dump(collection, f)

with open(join(metadata_save_dir, "metadata.geojson"), "r") as f:
    collection = geojson.load(f)    

In [ ]:
features = collection['features']
print(f"{len(features) = }")
set([feature['properties']['percent_covered'] for feature in features])

In [ ]:
download_imagery_dir = join(download_dir, "imagery", mosaic['name'])
os.makedirs(download_imagery_dir, exist_ok=True)

In [ ]:

def download_feature(feature):
    file_id = feature['properties']['id']
    file_path = join(download_imagery_dir, f"{file_id}.tif")
    
    download_url = feature['properties']['download']
    if exists(file_path):
        try:
            image = Image.open(file_path)
            assert image.size == (4096, 4096)
            return  # Skip if the file is already downloaded and is not corrupted
        except:
            print(f"Corrupted file: {file_path}.")

    response = requests.get(download_url, headers=headers)

    with open(file_path, "wb") as f:
        f.write(response.content)
        
    da = rxr.open_rasterio(file_path)
    da = da.assign_coords(x=np.round(da.x, 6))
    da = da.assign_coords(y=np.round(da.y, 6))
    kwargs = {'blockxsize': 512, 'blockysize': 512, 'tiled': True, 'compress': 'lzw', 'interleave': 'band'}
    da.rio.to_raster(file_path, **kwargs)

with cf.ThreadPoolExecutor(max_workers=48) as executor:
    list(tqdm(executor.map(download_feature, features), total=len(features)))

In [ ]:
#visualize the downloaded images
import rasterio
import matplotlib.pyplot as plt
data=rasterio.open('--.tif')## put tif file name
plt.imshow(data.read(1))